#### 음악 만들기 *with tensorflow* !!

In [3]:
#dependencies
from deepmusic.moduleloader import ModuleLoader
#predicts next key
from deepmusic.keyboardcell import KeyboardCell
#encapsulate song data so we can run get_scale, get_relative_methods
import deepmusic.songstruct as music

ModuleNotFoundError: No module named 'deepmusic'

In [2]:
#generate random numbers
import numpy as np
#for FLOWing
import tensorflow

In [ ]:
def build_network(self):
    #create computation graph, encapsulate session and the graph init
    input_dim = ModuleLoader.batch_builders.get_module().get_input_dim()
    
    #note data : format MIDI
    with tf.name_scope('placeholder_inputs'):
        self.inputs = [
            tf.placeholder(
            tf.float32, #numerical data
            [self.batch_size, input_dim], #how nuch data ( batch size )
            name='input'
            )
        ]
        
    #targets 88 key(piano), binary classification problem - key가 눌렸는지(1) 눌리지 않았는지(0)
    with tf.name_scope('placeholder_targets'):
        self.targets = [
            tf.placeholder(
            tf.int32, #0 or 1
            [self.batch_size],
            name='target')
        ]
        
    #hidden state
    with tf.name_scope('placeholder_use_prev'):
        self.use_prev = p[
            tf.placeholder(
            tf.bool,
            [],
            name='use_prev')
        ]
            
    #define our network
    self.loop_processing = ModuleLoader.loop_processings.build_module(self.args)
    def loop_rnn(prev, i):
        nex_input = self.loop_processing(prev)
        return tf.cond(self.prev[i], lambda: nex_input, lambda: self.inputs[i])
    
    #build seq2seq model
    self.outputs, self.final_state = tf.nn.seq2seq.rnn_decoder(
        decoder_inputs = self.inputs, #because defined in keyboard cell
        initial_state=None,
        cell=KeyboardCell,
        loop_function=loop_rnn
    )
    
    #training step
    #lets define our loss function : cross entropy -multi-classification
    
    loss_fct = tf.nn.seq2seq.seuence_loss(
    self.outputs,
    self.targets,
    softmax_loss_function=tf.nn.softmax.cross_entropy_with_logits,
    average_across_timesteps=True,
    average_across_batch=True
    )
    
    #initialize the optimizer, minimize the loss
    opt = tf.train.AdamOptimizer(
        learning_rate=self.current_learning_rate,
        betal=0.9,
        beta2=0.999,
        epsilon=1e-08
    )
    
    self.opt_op = opt.minimize(loss_fct)
    